In [ ]:
from numpy.random import seed
seed(1)

import tensorflow
tensorflow.random.set_seed(3)

from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from tensorflow.keras.activations import softmax

In [ ]:
import import_ipynb

from preprocessing import *

In [ ]:
def get_hybrid_fusion(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_hybrid_fusion_preprocessing(raw_data, seq_length=seq_length)
    
    feature_length = len(events_without_movement) + len(plot_names) + emb_dim
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(seq_length, feature_length,)))
    logs_responses.add(GRU(128, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs_responses.add(Dense(64, activation='relu'))
    logs_responses.add(Dropout(0.1))
    
    pretest_input = Input(shape=1)
    
    merged = Concatenate()([logs_responses.output, pretest_input])
    
    dense2 = Dense(8)(merged)
    dense3 = Dense(8)(merged)
    dense4 = Dense(8)(dense3)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense2)
    
    combined_model = Model([logs_responses.input, pretest_input], 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.01)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_hybrid_fusion_new(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_hybrid_fusion_preprocessing(raw_data, seq_length=seq_length)
    
    feature_length = len(events_without_movement) + len(plot_names) + emb_dim
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(seq_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs_responses.add(Dropout(0.2))
    logs_responses.add(Dense(1, activation='sigmoid'))
    
    pretest_input = Input(shape=1)
    
    merged = Concatenate()([logs_responses.output, pretest_input])
    
    dense1 = Dense(64, activation='relu')(merged)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense4 = Dense(8, activation='sigmoid')(dense2)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model([logs_responses.input, pretest_input], 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_hybrid_fusion_prev(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_hybrid_fusion_preprocessing(raw_data, seq_length=seq_length)
    
    feature_length = len(events_without_movement) + len(plot_names) + emb_dim
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(seq_length, feature_length,)))
    logs_responses.add(GRU(128, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs_responses.add(Dropout(0.1))
    logs_responses.add(Dense(1, activation='sigmoid'))
    
    pretest_input = Input(shape=1)
    
    merged = Concatenate()([logs_responses.output, pretest_input])
    
    dense1 = Dense(64, activation='relu')(merged)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense2)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model([logs_responses.input, pretest_input], 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_early_fusion_mtl(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_early_fusion_preprocessing(raw_data, seq_length=seq_length)
    
    feature_length = len(events_without_movement) + len(plot_names) + emb_dim + 1
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(seq_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs_responses.add(Dropout(0.2))
    logs_responses.add(Dense(1, activation='sigmoid'))
    
    dense1 = Dense(64, activation='relu')(logs_responses.output)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense4 = Dense(8, activation='sigmoid')(dense2)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model(logs_responses.input, 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_late_fusion_mtl(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_late_fusion_preprocessing(raw_data, seq_length=seq_length)
    
    actions_feature_length = len(events_without_movement) + len(plot_names)
    responses_feature_length = emb_dim
    logs = Sequential()
    logs.add(InputLayer(input_shape=(seq_length, actions_feature_length,)))
    logs.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs.add(Dropout(0.2))
    logs.add(Dense(1, activation='sigmoid'))
    
    responses = Sequential()
    responses.add(InputLayer(input_shape=(seq_length, responses_feature_length,)))
    responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    responses.add(Dropout(0.2))
    responses.add(Dense(1, activation='sigmoid'))
    
    pretest = Sequential()
    pretest.add(InputLayer(input_shape=(1)))
    pretest.add(Dense(1, activation='sigmoid'))
    
    merged = Concatenate()([logs.output, responses.output, pretest.output])
    
    dense1 = Dense(64, activation='relu')(merged)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense4 = Dense(8, activation='sigmoid')(dense2)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model([logs.input, responses.input, pretest.input], 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_pretest_mtl(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_pretest_preprocessing(raw_data, seq_length=seq_length)
    
    pretest = Sequential()
    pretest.add(InputLayer(input_shape=(1)))
    pretest.add(Dense(1, activation='sigmoid'))
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(pretest.output)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(pretest.output)
    
    combined_model = Model(pretest.input, 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_reflection_mtl(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_reflection_preprocessing(raw_data, seq_length=seq_length)
    
    responses_feature_length = emb_dim
    
    responses = Sequential()
    responses.add(InputLayer(input_shape=(seq_length, responses_feature_length,)))
    responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    responses.add(Dropout(0.2))
    responses.add(Dense(1, activation='sigmoid'))
    
    dense1 = Dense(64, activation='relu')(responses.output)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense4 = Dense(8, activation='sigmoid')(dense2)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model(responses.input, 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_reflection_pretests_mtl(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_reflection_pretest_preprocessing(raw_data, seq_length=seq_length)
    
    responses_feature_length = emb_dim
    
    responses = Sequential()
    responses.add(InputLayer(input_shape=(seq_length, responses_feature_length,)))
    responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    responses.add(Dropout(0.2))
    responses.add(Dense(1, activation='sigmoid'))
    
    pretest_input = Input(shape=1)
    
    merged = Concatenate()([responses.output, pretest_input])
    
    dense1 = Dense(64, activation='relu')(merged)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense4 = Dense(8, activation='sigmoid')(dense2)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model([responses.input, pretest_input], 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_logs_pretests_mtl(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_logs_pretest_preprocessing(raw_data, seq_length=seq_length)
    
    actions_feature_length = len(events_without_movement) + len(plot_names)
    
    logs = Sequential()
    logs.add(InputLayer(input_shape=(seq_length, actions_feature_length,)))
    logs.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs.add(Dropout(0.2))
    logs.add(Dense(1, activation='sigmoid'))
    
    pretest_input = Input(shape=1)
    
    merged = Concatenate()([logs.output, pretest_input])
    
    dense1 = Dense(64, activation='relu')(merged)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense4 = Dense(8, activation='sigmoid')(dense2)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model([logs.input, pretest_input], 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_logs_responses_mtl(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_logs_responses_preprocessing(raw_data, seq_length=seq_length)
    
    feature_length = len(events_without_movement) + len(plot_names) + emb_dim
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(seq_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs_responses.add(Dropout(0.2))
    logs_responses.add(Dense(1, activation='sigmoid'))
    
    dense1 = Dense(64, activation='relu')(logs_responses.output)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense4 = Dense(8, activation='sigmoid')(dense2)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model(logs_responses.input, 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_logs_mtl(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = get_logs_preprocessing(raw_data, seq_length=seq_length)
    
    actions_feature_length = len(events_without_movement) + len(plot_names)
    
    logs = Sequential()
    logs.add(InputLayer(input_shape=(seq_length, actions_feature_length,)))
    logs.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs.add(Dropout(0.2))
    logs.add(Dense(1, activation='sigmoid'))
    
    dense1 = Dense(64, activation='relu')(logs.output)
    dense2 = Dense(8, activation='sigmoid')(dense1)
    dense4 = Dense(8, activation='sigmoid')(dense2)
    dense3 = Dense(8, activation='sigmoid')(dense1)
    
    output_posttest = Dense(1, activation='sigmoid', name='posttest_label')(dense4)
    output_rating = Dense(1, activation='sigmoid', name='rating_label')(dense3)
    
    combined_model = Model(logs.input, 
                           outputs=[output_posttest, output_rating])
    
    optimizer = Adam(lr=0.001)
    
    losses = {"posttest_label": 'mse',
              "rating_label": 'mse'}
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss=losses) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids